In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
from sklearn.preprocessing import StandardScaler
import joblib
import os

data = pd.read_csv("dataset/pluvial_flood_data_balanced.csv")
X = data.drop(columns=["pluvial_binary"]).astype("float32")
y = data["pluvial_binary"].astype(np.float32)

print(f"{len(X)} samples | Flood: {y.sum()} | No Flood: {(y == 0).sum()}")

3300 samples | Flood: 1800.0 | No Flood: 1500


In [2]:
def generate_pluvial_trust(row):
    rain = row["rainfall_intensity"]
    urban = row["urbanization_index"]
    impervious = row["impervious_ratio"]
    drainage = row["drainage_density"]
    convergence = row["convergence_index"]

    rain_factor       = np.clip((rain - 40) / 80, 0, 1)
    urban_factor      = np.clip((urban - 0.4) / 0.6, 0, 1)
    impervious_factor = np.clip((impervious - 0.4) / 0.6, 0, 1)
    drainage_factor   = 1.0 - np.clip((drainage - 2.5) / 2.5, 0, 1)
    convergence_factor= np.clip((convergence - 0.5) / 0.5, 0, 1)

    composite = (
        0.3 * rain_factor +
        0.25 * urban_factor +
        0.2 * impervious_factor +
        0.15 * convergence_factor +
        0.1 * drainage_factor
    )
    return 0.5 + composite

trust_labels = X.apply(generate_pluvial_trust, axis=1).values.astype(np.float32)

In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
@register_keras_serializable()
def floodtrust_activation(x):
    return 0.5 + tf.sigmoid(x)

input_layer = layers.Input(shape=(X.shape[1],))
z = layers.Dense(32, activation="relu")(input_layer)
z = layers.Dense(16, activation="relu")(z)
z = layers.Dense(8, activation="relu")(z)
output = layers.Dense(1, activation=floodtrust_activation)(z)

trustnet = models.Model(inputs=input_layer, outputs=output)
trustnet.compile(optimizer="adam", loss="mse")
trustnet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
trustnet.fit(X_scaled, trust_labels, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0343 - val_loss: 0.0039
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0029 - val_loss: 0.0013
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 7.4062e-04
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.1746e-04 - val_loss: 5.2625e-04
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.1438e-04 - val_loss: 4.0250e-04
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.0734e-04 - val_loss: 3.4712e-04
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.6451e-04 - val_loss: 2.9299e-04
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.0798e-04 - val_loss: 2.6600e-04
Epoch 9/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9748e-04 - val_loss: 2.6847e-04
Epoch 10/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 2.4264e-04 - val_loss: 2.2347e-04


In [6]:
trustnet.save("models/PV-FloodTrustNet.h5")
joblib.dump(scaler, "models/PV-floodtrust_scaler.pkl")

['models/PV-floodtrust_scaler.pkl']